## Name : Hieu Nguyen - Joshua Blaz


Setup the required library and connect to database

In [2]:
import pandas as pd
import sqlalchemy as sa
import json

In [3]:
%load_ext sql

In [4]:
def getCreds(filename, subset, defaults = {}):
    """ Use `filename` to look for a file containing a json-encoded dictionary
        of credentials.  If the file is successfully found and contains valid
        json, return the sub-dictionary based on `subset`.  If the file is not
        found, is not accessible, has improper encoding, or if the subset is 
        not present in the dictionary, return the given defaults.
    """
    try:
        with open(filename, 'r') as file:
            D = json.load(file)
            if D[subset]:
                return D[subset]
            else:
                return defaults
    except:
        return defaults

creds = getCreds("creds.json", "mysql", defaults={'user': 'studen_j1', 
                                                  'password': 'studen_j1'})
def db_setup(user, password, database):
    template = 'mysql+mysqlconnector://{}:{}@hadoop2.mathsci.denison.edu/{}'
    cstring = template.format(user, password, database)
    e = sa.create_engine(cstring)
    c = e.connect()
    return e, c, cstring

In [5]:
try:
    connection.close()
    del engine
except:
    pass
database = creds['user']
engine, connection, cstring = db_setup(creds['user'], creds['password'], database)

In [11]:
imdb = "USE imdb"
connection.execute(imdb)

First query for analysis

In [21]:
def countGenre(db_connection):
    """
    This function will run the SQL query that give us a table in which each observation is a genre and its total count of movies
    Parameter: conn: an sqlalchemy connection
    Return: df: a pandas dataframe of the data
    """

    query = """
    SELECT genre, COUNT(primaryTitle) AS Total_Movies_Number
    FROM titles INNER JOIN principles USING(titleID)
    INNER JOIN names USING(nameID)
    INNER JOIN ratings USING(titleID)
    INNER JOIN genres USING(titleID)
    GROUP BY genre DESC
    """
    
    resultproxy = db_connection.execute(query)
    resultset = resultproxy.fetchall()
    
    # Construct the pandas dataframe from the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [22]:
#run the function to get a pandas dataframe and export it to csv file for further analysis purpose
count_Genre = countGenre(connection)
count_Genre.to_csv('countGenre.csv')

Second query for analysis

In [51]:
def getGenreCount_byYear(db_connection):
    """
    This function will run the query that give us a summary table of year and genre, followed by the number of
    movies and the average rating for each observation
    Parameter: conn: an sqlalchemy connection
    Return: df: a pandas dataframe of the data
    """
    # Set up the SQL query: In this query, we pull out the genre and startYear columns by SELECT and do aggregation on the 
    #                       number of movies and the average rating for each observation using AVG and COUNT, respectively. 
    #                       In the end, we are interested in 5 genres so we picked them out using HAVING operator to see how the trends of count look like.
    #                       The data table was created by a series of INNER JOIN on the nameID and titleID columns.
    
    
    query = """SELECT genre, AVG(averageRating), startYear, COUNT(*)
                  FROM titles INNER JOIN principles USING(titleID)
                  INNER JOIN names USING(nameID)
                  INNER JOIN ratings USING(titleID)
                  INNER JOIN genres USING(titleID)
                  GROUP BY startYear, genre
                  HAVING genre = 'Drama' OR genre = 'Romance' OR genre = 'Thriller' OR genre = 'Action' OR genre = 'Comedy'
                  """
    
    #execute the query
    resultproxy = db_connection.execute(query)
    resultset = resultproxy.fetchall()
    
    # Construct the pandas dataframe from the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [52]:
#run the function to get a pandas dataframe and export it to csv file for further analysis purpose
countGenre_byYear = getGenreCount_byYear(connection)
countGenre_byYear.to_csv('getGenreCount_byYear.csv')

Third query for analysis

In [47]:
def actor_actressCount(db_connection):
    """
    This function will run the query that give us a table which each observation is Year and category (actor or actress)
    and then aggregate on the Count of each individual year / category
    Parameter: conn: an sqlalchemy connection
    Return: df: a pandas dataframe of the data
    """
    
    # Set up the SQL query: In this query, we pull out category - Year and group them by both startYear and category 
    #                       and then aggregate on the count of each individual observation
    
    query = """
    SELECT category, startYear, COUNT( *) as Count
    FROM
    (SELECT category, titleID
    FROM principles
    WHERE category = 'actor' or category = 'actress') AS sub1
    JOIN titles USING(titleID)
    GROUP BY startYear, category
    """
    
    #execute the query
    resultproxy = db_connection.execute(query)
    resultset = resultproxy.fetchall()
    
    # Construct the pandas dataframe from the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [48]:
countActorActress = actor_actressCount(connection)
countActorActress.to_csv('countActorActress.csv')

,category,startYear,Count
0,actor,1893.0,2
1,actor,1894.0,2
2,actor,1895.0,2
3,actress,1895.0,1
4,actor,1896.0,9
5,actress,1896.0,3
6,actor,1898.0,2
7,actress,1898.0,1
8,actor,1899.0,2
9,actress,1899.0,3


Fourth query for analysis

In [15]:
def GoT(db_connection):
    """
    This function will run the query that give us a table of main actor and actress ID on Game of Throne
    Parameter: conn: an sqlalchemy connection
    Return: df: a pandas dataframe of the data
    """
    
    # Set up the SQL query: In this query, we pull out the nameID by SELECT operator from a sub table, in which we refine
    #                       the category down to actor and actress only for the code efficiency. We later filter out
    #                       Game Of Thrones actor and actress by the LIKE operator                       
    
    query = """
    SELECT nameID
    FROM
    (SELECT nameID, primaryName, primaryTitle, averageRating, numVotes
    FROM
    (SELECT titleID, category, nameID
    FROM principles
    WHERE category = 'actor' OR category = 'actress') as att
    JOIN names USING(nameID)
    JOIN titles USING(titleID)
    JOIN ratings USING(titleID)
    WHERE primaryTitle LIKE 'Game of Thrones' AND startYear = 2011) as GOT
    """
    
    #execute the query
    resultproxy = db_connection.execute(query)
    resultset = resultproxy.fetchall()
    
    # Construct the pandas dataframe from the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [17]:
GoTactor = GoT(connection)
GoTactor

,nameID
0,nm0182666
1,nm0227759
2,nm0322513
3,nm0372176
4,nm3229685
5,nm3586035
6,nm3592338
7,nm3849842


In [18]:
def RatingGOT(db_connection):
    """
    This function will give us the summary table in which each observation is Game of Throne's main actor and actress
    and their movies average rating with and without Game Of Throne
    Parameter: conn: an sqlalchemy connection
    Return: df: a pandas dataframe of the data
    """
    
    # Set up the SQL query: This query is divided into two parts where the first subquery give us the summary table of 
    #                       all Game Of Throne actor movies' average rating with Game of Throne and the second subquery give us the summary table of 
    #                       all Game Of Throne actor movies' average rating without Game of Throne. We then join the two table together
    #                       to get own desired table.
    #
    query = """
    SELECT * FROM
    (SELECT primaryName, AVG(averageRating) as AVG_WithGOT
    FROM principles
    JOIN titles USING(titleID)
    JOIN ratings USING(titleID)
    JOIN names USING(nameID)
    WHERE nameID IN ('nm0182666', 'nm0227759', 'nm0322513' ,'nm0372176', 'nm3229685', 'nm3586035', 'nm3592338' ,'nm3849842')
    GROUP BY primaryName) as withGOT
    JOIN
    (SELECT AVG_WithoutGOT, primaryName
    FROM
    (SELECT DISTINCT primaryTitle, averageRating as AVG_WithoutGOT, primaryName
    FROM principles
    JOIN titles USING(titleID)
    JOIN genres USING(titleID)
    JOIN ratings USING(titleID)
    JOIN names USING(nameID)
    WHERE nameID IN ('nm0182666', 'nm0227759', 'nm0322513' ,'nm0372176', 'nm3229685', 'nm3586035', 'nm3592338' ,'nm3849842')
    AND primaryTitle NOT LIKE 'Game of Thrones'
    GROUP BY primaryName) as subsub) as withoutGOT
    USING(primaryName)
    """
    
    #execute the query
    resultproxy = db_connection.execute(query)
    resultset = resultproxy.fetchall()
    
    # Construct the pandas dataframe from the result
    column_names = resultproxy.keys()
    df = pd.DataFrame(resultset, columns=column_names)
    return df

In [54]:
#run the function to get a pandas dataframe and export it to csv file for further analysis purpose
GoT_Rating = RatingGOT(connection)
GoT_Rating.to_csv('FinalGOT.csv')

#                 Project 2: IMDB Database<br>
## Hieu Nguyen - Joshua Blaz

## Introduction

In this project, we worked with the IMDB data base, which is a set of tables on different movies data. Each of this table has a primary key and is conformed to third normal form constraints, which state that each row represents exactly one observation and each column represents exactly one variable. The set of tables also follow a set of relationship using their primary key. We began this project with a set of questions that might be answered by using database: According to the IMDB data, what types of movies are the most popular? Then, how did the genre's popularity develop to present day? We moved on to the next realm of analysis, which is the questions on the demographic data of movies. Specifically, we want to explore the difference in number of actors and actresses by year to see if there is a significant difference between two numbers. Lastly, we questioned that some actors and actresses have a "one-hit wonder" that make them successful and lead to the high rating among all of their movies. Using Game of Thrones, one of the most famous series in recent years, we are going to examine the difference in the these actors and actresses movies rating with and without Game of Thrones.

## Popularity of Genre

 This is a bar graph displaying the number of movies in each genre of all time plotted against one another. This graph showed us, in terms of genre, how popular drama is with a count of 221,702 movies. There are by far more drama movies on IMDB than any other genre. In fact, the drama genre leads the second most popular genre,comedy, by a count of nearly 73,000 titles. This gaping differential in genre popularity initially took us by surprise, but it actually makes sense, as by inspection, we realized just how many episodes there are of random day-time soap operas and other drama shows. Given that the IMDB database contains both movies and TV shows and even some video games, this differential can definitely be accounted for  by the presence of all of those drama TV shows. 


![Genre](GenreCount.png)

## Development Of Genre Over Time

   After noticing how much more common the drama genre was than any other genre, we decided to explore this a little further. We want to explore the popularity of genre by year to see if there is any change in people's preference by year. <br>
   This line plots show the development of genre popularity by year, as each line represent a genre and the count of movies. As we can see, Comedy surprisingly appears to be the first genre developed in 1890 and other genre started to appear in the 1900s to make a collection of different genres for the movie industries. Drama started to overwhelm other genres in its movies count by the 1920s. At some period of time like 1980s and 1990s, Comedy catched up and exceeded Drama in the movies count. However, since the 2000s, number of Drama movies has been growing exponentially with a steep slope of line. We can also see the declining of Comedy and the raising of Action in recent years
   
   

![GenreCountByYear](genrecountbyyear.png)

## Difference In Gender Of Actor

Gender differentiation has always been an issues in any fields of life. Therefore, in this part, using the IMDB database, we want to explore on the gender difference in the filming industries specifically. <br>
The graph below shows us the difference between number of actor and actress in the industry. Male was the gender that initially took the main parts in the industry in 1910s. It was in 1930s when there was emergence in the number of actress in the industry. However, as we can see, the blue area, indicating the number of male actor, always double the pink area, indicating the number of female actors. This indicate a significant difference in the number of actors and actresses over time, as actor's count has always been overwhelmed that of actress. Further research on theater history or woman studies should take this phenomenal demographical difference into account to examine the reasoning behind this.

![ActorActressCount](ActorActressCount.png)

## Game Of Throne!

For the last part of this project, we used Game Of Throne, one of the most popular series in recent years, to provide a case study for our hypothesis: Is it true that some actors have a "one- hit wonder" movie that overwhelm every other movies of them? <br>
This side-by-side bar chart shows the average rating of seven Game of Throne's main actor's movies with and without Game Of Throne rating. For all seven actors, the average rating with Game of Throne far exceeded the average rating without Game Of Throne. For example, Emilia Clarke's average rating with Game Of Throne included is 8.9 / 10, whereas, her average rating is only 6.5 / 10 with Game Of Throne exclude.  <br>


![GoT](GoT.png)